In [1]:
from datasets import load_dataset

In [2]:
raw_dataset = load_dataset('kde4',lang1='en',lang2='zh_CN')

Using the latest cached version of the module from /Users/liuchu/.cache/huggingface/modules/datasets_modules/datasets/kde4/243129fb2398d5b0b4f7f6831ab27ad84774b7ce374cf10f60f6e1ff331648ac (last modified on Tue Dec 31 15:44:07 2024) since it couldn't be found locally at kde4, or remotely on the Hugging Face Hub.


In [5]:
split_dataset = raw_dataset['train'].train_test_split(train_size=0.9,seed=20)

In [6]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 125699
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 13967
    })
})

In [11]:
split_dataset['train'][7886]['translation']

{'en': 'Username:', 'zh_CN': '用户名 ：'}

In [12]:
from transformers import AutoTokenizer

In [13]:
model_checkpoint = 'Helsinki-NLP/opus-mt-en-zh'

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

/Users/liuchu/opt/anaconda3/envs/mem0/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [16]:
tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-zh', vocab_size=65001, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	65000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [28]:
en_sentence = split_dataset['train'][3]['translation']['en']

In [29]:
zh_sentence = split_dataset['train'][3]['translation']['zh_CN']

In [30]:
inputs = tokenizer(en_sentence,text_target=zh_sentence)

In [34]:
inputs

{'input_ids': [26, 13932, 49644, 36, 17, 3778, 12179, 13, 39382, 1857, 15, 13, 816, 269, 6, 84, 32, 3, 471, 35, 3, 1963, 27139, 131, 26953, 7866, 3778, 6, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [453, 18437, 9470, 1401, 22, 17, 8, 35797, 3793, 673, 3300, 4993, 12, 32891, 19543, 3278, 10, 11560, 35797, 67, 1963, 2926, 1333, 131, 228, 18437, 9470, 1401, 8, 35797, 5051, 8, 10, 0]}

In [35]:
''.join(tokenizer.convert_ids_to_tokens(inputs['labels']))

'▁STRING▁()▁函数返回给定数字的字符串值。▁此函数与▁NUM2STRING▁函数相同▁。</s>'

In [36]:
####### 手动实现transformer

In [37]:
from torch import nn

In [48]:
class FeedForward(nn.Module):
    
    def __init__(self,input_dim,hidden_dim,output_dim):
        super().__init__()
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(input_dim,hidden_dim)
        self.linear2 = nn.Linear(hidden_dim,output_dim)
        
    
    def forward(self,x):
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [49]:
import torch

In [50]:
x = torch.randn((4,5))

In [52]:
fd = FeedForward(5,7,6)

In [54]:
fd(x).shape

torch.Size([4, 6])

In [67]:
import torch.nn.functional as F

In [59]:
class LayerNorm(nn.Module):
    
    def __init__(self,input_dim):
        super().__init__()
        self.ln = nn.LayerNorm(input_dim)

    
    def forward(self,x):
        return self.ln(x)

In [66]:
x = torch.randn(5,4)
ln = LayerNorm(4)
ln(x)

tensor([[ 1.3836, -1.3330,  0.3668, -0.4173],
        [ 0.9285, -0.2232,  0.8387, -1.5441],
        [ 0.8393,  0.0920,  0.7285, -1.6599],
        [ 1.5067, -0.0065, -0.2004, -1.2999],
        [-0.7406, -0.4775,  1.7229, -0.5048]],
       grad_fn=<NativeLayerNormBackward0>)

In [115]:
class Attention(nn.Module):
    
    def __init__(self,input_dim,hidden_dim):
        super().__init__()
        self.qw = nn.Linear(input_dim,hidden_dim)
        self.kw = nn.Linear(input_dim,hidden_dim)
        self.vw = nn.Linear(input_dim,hidden_dim)
    
    def forward(self,x):
        ## B,T,C
        B,T,C = x.shape
        q = self.qw(x)
        k = self.kw(x)
        v = self.vw(x)
        print(q.shape,k.shape,k.T.shape)
        att = q @ k.permute(0,2,1)
#         att = att.masked_fill(mask, value)
        att = F.softmax(att,dim=-1)
        v = att @ v
        return v

In [116]:
x = torch.randn(5,3,4)

In [117]:
att = Attention(4,6)

In [118]:
att(x).shape

torch.Size([5, 3, 6]) torch.Size([5, 3, 6]) torch.Size([6, 3, 5])


torch.Size([5, 3, 6])

In [760]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self,input_dim,head_size,hidden_size):
        super().__init__()        
        self.head_size = head_size
        self.hidden_size = hidden_dim
        self.qw = nn.Linear(input_dim,head_size * hidden_dim)
        self.kw = nn.Linear(input_dim,head_size * hidden_dim)
        self.vw = nn.Linear(input_dim,head_size * hidden_dim)
        
    def forward(self,q,k,v,masked=False):
        #### q ==> B,T,C
        q = self.qw(q)
        k = self.kw(k)
        v = self.vw(v)
        #### q ===> B,head_size,T,hidden_size
        B,T,C = q.shape
        q = q.reshape(B,T,self.head_size,self.hidden_size).permute(0,2,1,3)
        B,T,C = k.shape
        k = k.reshape(B,T,self.head_size,self.hidden_size).permute(0,2,1,3)
        B,T,C = v.shape
        v = v.reshape(B,T,self.head_size,self.hidden_size).permute(0,2,1,3)
        B,head_size,T,hidden_size = q.shape
        att = q @ k.permute(0,1,3,2) # B,head_size,T,T
        if masked:
            _,_,m,n = att.shape
            mask = torch.ones(m,n)
            mask = torch.tril(mask)
            att = att.masked_fill(mask==0,float('-inf'))
        att = F.softmax(att,dim=-1)
        v = att @ v  # B,head_size,T,hidden_size
        v = v.permute(0,2,1,3) # B,T,head_size,hidden_size
        v = v.reshape(B,T,self.head_size * self.hidden_size)
        return v       
        

In [761]:
x = torch.randn(5,4,3)

In [762]:
att1 = MultiHeadAttention(3,2,3)

In [763]:
att = torch.randn(1,1,2,4)

In [764]:
att

tensor([[[[ 1.2754, -1.5449,  0.9508, -0.7407],
          [ 0.6911,  0.5579, -0.8990, -1.2444]]]])

In [765]:
 _,_,m,n = att.shape
mask = torch.ones(m,n)

In [766]:
mask = torch.tril(mask)

In [767]:
mask == 0

tensor([[False,  True,  True,  True],
        [False, False,  True,  True]])

In [768]:
att = att.masked_fill(mask==0,float('-inf'))

In [769]:
torch.softmax(att,dim=-1)

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000],
          [0.5333, 0.4667, 0.0000, 0.0000]]]])

In [770]:
att(x,x,x).shape

TypeError: 'Tensor' object is not callable

In [771]:
class EncoderBlock(nn.Module):
    
    def __init__(self,input_dim,head_size,hidden_dim):
        super().__init__()
        self.mha = MultiHeadAttention(input_dim,head_size,hidden_dim)
        self.ln1 = LayerNorm(input_dim)
        self.fd = FeedForward(input_dim,hidden_dim,input_dim)
        self.ln2 = LayerNorm(input_dim)
    
    def forward(self,q,k,v):
        x = q + self.mha(q,k,v)
        x = self.ln1(x)
        x = x + self.fd(x)
        x = self.ln2(x)
        return x

In [772]:
block = EncoderBlock(4,2,2)

In [773]:
x = torch.randn(5,3,4)

In [894]:
class DecoderBlock(nn.Module):
    
    def __init__(self,input_dim,head_size,hidden_dim):
        super().__init__()
        self.mha = MultiHeadAttention(input_dim,head_size,hidden_dim)
        self.ln1 = LayerNorm(input_dim)
        self.fd = FeedForward(input_dim,hidden_dim,input_dim)
        self.ln2 = LayerNorm(input_dim)
        self.mha2 = MultiHeadAttention(input_dim,head_size,hidden_dim)
        self.fd2 = FeedForward(input_dim,hidden_dim,input_dim)
        self.ln3 = LayerNorm(input_dim)
        
    def forward(self,x,k,v):
#         x,k0,v0 = self.mha.qkv(x) ### 需要masked
        x = x + self.mha(x,k,v)
        x = self.ln1(x)
        x = x + self.mha2(x,k,v) ### cross attention
        x = self.ln2(x)
        x = x + self.fd2(x)
        x = self.ln3(x)
        return x

In [895]:
class Transformer(nn.Module):
    
    def __init__(self,n,input_dim,head_size,hidden_dim,input_vocab_size,output_vocab_size):
        super().__init__()
        self.encoder_blocks = nn.ModuleList(
           [EncoderBlock(input_dim,head_size,hidden_dim) for _ in range(n)]
        )
        self.decoder_blocks = nn.ModuleList(
           [DecoderBlock(input_dim,head_size,hidden_dim)   for _ in range(n)]
        )
        self.input_embeddings = nn.Embedding(input_vocab_size,input_dim)
        self.output_embeddings = nn.Embedding(output_vocab_size,input_dim)
        self.output_linear = nn.Linear(head_size * hidden_dim,output_vocab_size)
        self.input_pos_embedding = nn.Embedding(1024,input_dim)
        self.output_pos_embedding = nn.Embedding(1024,input_dim)
        

    def forward(self,x,y):
        #### x ==> B,T
        B,T = x.shape
        x = self.input_embeddings(x) ### B,T,C
        x_pos = self.input_pos_embedding(torch.arange(T))
        x = x + x_pos
        for block in self.encoder_blocks:
            x = block(x,x,x)  ### B,head_size,T,hidden_size
        B,T = y.shape
        y = self.output_embeddings(y) ### B,T,C
        y_pos = self.output_pos_embedding(torch.arange(T))
        y = y + y_pos
        for block in self.decoder_blocks:
            y = block(y,x,x) ### B,head_size,T,hidden_size
        logits = self.output_linear(y) # B,T,output_vocab_size
        return logits
    

In [896]:
n = 5
input_dim = 4
head_size = 2
hidden_dim = input_dim // head_size
input_vocab_size = 10
output_vocab_size = 15

In [897]:
transformer = Transformer(n,input_dim,head_size,hidden_dim,input_vocab_size,output_vocab_size)

In [898]:
x = torch.LongTensor([
    [0,1,3],
    [0,2,3]
])
y = torch.LongTensor([
    [1,3,4,5],
    [2,3,4,6]
])

In [899]:
transformer(x,y).shape

torch.Size([2, 4, 15])

In [900]:
dataset = [
    {"input":"In the heart of the city, there is a beautiful park where people can enjoy nature and relax after a long day of work.","output":"在城市的中心，有一座美丽的公园，人们可以在那里享受自然，在漫长的工作日后放松身心。"},
    {"input":"The rapid development of technology has brought about significant changes in our daily lives, making it more convenient for us to communicate and access information.","output":"科技的快速发展给我们的日常生活带来了巨大的变化，使我们能够更方便地进行沟通和获取信息。"},
    {"input":"Despite the challenges we face, we should always maintain an optimistic attitude towards life, believing that every problem has a solution.","output":"尽管我们面临挑战，但我们应该始终保持对生活的乐观态度，相信每个问题都有解决方案。"},
    {"input":"With the increasing popularity of online shopping, more and more people are choosing to purchase goods through the internet, which not only saves time but also offers a wider range of choices.","output":"随着网上购物的日益普及，越来越多的人选择通过互联网购买商品，这不仅节省了时间，还提供了更广泛的选择。"}
]


In [901]:
class Tokenizer:
    
    def __init__(self,sentences):
        self.vocab_set = set()
        for sentence in sentences:
            self.vocab_set.update(sentence)
        self.vocab_set = list(self.vocab_set)
        self.vocab_set = ['<pad>','<bos>','<eos>'] + self.vocab_set
        self.token2id = {c:i for i,c in enumerate(self.vocab_set)}
        self.id2token = {i:c for c,i in self.token2id.items()}
    
    def convert_token_to_id(self,tokens):
        return [self.token2id.get(t,'') for t in tokens]
    
    def convert_id_to_token(self,ids):
        return [self.id2token.get(i,-1) for i in ids]

In [902]:
[d['input'] for d in dataset]

['In the heart of the city, there is a beautiful park where people can enjoy nature and relax after a long day of work.',
 'The rapid development of technology has brought about significant changes in our daily lives, making it more convenient for us to communicate and access information.',
 'Despite the challenges we face, we should always maintain an optimistic attitude towards life, believing that every problem has a solution.',
 'With the increasing popularity of online shopping, more and more people are choosing to purchase goods through the internet, which not only saves time but also offers a wider range of choices.']

In [903]:
input_tokenizer = Tokenizer([d['input'] for d in dataset])

In [904]:
output_tokenizer = Tokenizer([d['output'] for d in dataset])

In [905]:
output_tokenizer.convert_id_to_token([0,1,2,3,4,5])

['<pad>', '<bos>', '<eos>', '每', '后', '更']

In [906]:
def process(sentences,tokenizer,max_length,is_output=False):
    res = []
    for sentence in sentences:
        arr = tokenizer.convert_token_to_id(sentence)
        if is_output:
            arr = tokenizer.convert_token_to_id(['<bos>']) + arr + tokenizer.convert_token_to_id(['<eos>'])
        if len(arr) > max_length:
            arr = arr[:max_length]
        else:
            arr = arr + tokenizer.convert_token_to_id(['<pad>']) * (max_length - len(arr))
        res.append(arr)
    return res

In [907]:
inputs = [d['input'] for d in dataset]

In [908]:
inputs

['In the heart of the city, there is a beautiful park where people can enjoy nature and relax after a long day of work.',
 'The rapid development of technology has brought about significant changes in our daily lives, making it more convenient for us to communicate and access information.',
 'Despite the challenges we face, we should always maintain an optimistic attitude towards life, believing that every problem has a solution.',
 'With the increasing popularity of online shopping, more and more people are choosing to purchase goods through the internet, which not only saves time but also offers a wider range of choices.']

In [909]:
input_tokenizer.convert_token_to_id(['<pad>'])

[0]

In [910]:
inputs = process([d['input'] for d in dataset],input_tokenizer,30)

In [911]:
outputs = process([d['output'] for d in dataset],output_tokenizer,30,True)

In [912]:
x = torch.LongTensor(inputs)

In [913]:
y = torch.LongTensor(outputs)

In [928]:
n = 10
input_dim = 1600
head_size = 40
hidden_dim = input_dim // head_size
input_vocab_size = len(input_tokenizer.id2token)
output_vocab_size = len(output_tokenizer.id2token)

In [929]:
output_vocab_size

129

In [930]:
input_vocab_size

34

In [931]:
transformer = Transformer(n,input_dim,head_size,hidden_dim,input_vocab_size,output_vocab_size)

In [932]:
transformer(x,y).shape

torch.Size([4, 30, 129])

In [933]:
from torch.optim import AdamW

In [934]:
optim = AdamW(transformer.parameters(),lr=1e-3)

In [935]:
# 创建交叉熵损失函数
criterion = nn.CrossEntropyLoss()

In [936]:
for _ in range(1000):
    y_inputs = y[:,:-1]
    y_targets = y[:,1:]
    logits = transformer(x,y_inputs)
    B,T = y_targets.shape
    # 计算损失
    loss = criterion(logits.reshape(B*T,-1), y_targets.reshape(B*T))
    print(loss)
    
    optim.zero_grad()
    loss.backward()
    optim.step()
    

tensor(5.0338, grad_fn=<NllLossBackward0>)
tensor(6.7059, grad_fn=<NllLossBackward0>)
tensor(7.1730, grad_fn=<NllLossBackward0>)
tensor(6.0553, grad_fn=<NllLossBackward0>)
tensor(5.3269, grad_fn=<NllLossBackward0>)
tensor(4.8729, grad_fn=<NllLossBackward0>)
tensor(4.3969, grad_fn=<NllLossBackward0>)
tensor(7.6215, grad_fn=<NllLossBackward0>)
tensor(4.7034, grad_fn=<NllLossBackward0>)
tensor(4.7398, grad_fn=<NllLossBackward0>)
tensor(4.6002, grad_fn=<NllLossBackward0>)
tensor(4.4396, grad_fn=<NllLossBackward0>)
tensor(4.3256, grad_fn=<NllLossBackward0>)
tensor(4.2670, grad_fn=<NllLossBackward0>)
tensor(4.2502, grad_fn=<NllLossBackward0>)
tensor(4.2205, grad_fn=<NllLossBackward0>)
tensor(4.1258, grad_fn=<NllLossBackward0>)
tensor(4.0079, grad_fn=<NllLossBackward0>)
tensor(3.9277, grad_fn=<NllLossBackward0>)
tensor(3.8923, grad_fn=<NllLossBackward0>)
tensor(3.8890, grad_fn=<NllLossBackward0>)
tensor(3.9150, grad_fn=<NllLossBackward0>)
tensor(3.9494, grad_fn=<NllLossBackward0>)
tensor(3.94

tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.7888, grad_fn=<NllLossBackward0>)
tensor(3.78

KeyboardInterrupt: 

In [937]:
def predict(model,inputs):
    ids = input_tokenizer.convert_token_to_id(inputs)
    x = torch.LongTensor([ids])
    y = torch.LongTensor([output_tokenizer.convert_token_to_id(['<bos>'])])
    for _ in range(100):
        logits = model(x,y)
        ### logits B,T,vocab_size
        logits = logits[:,-1,:]
        ### logits B,T,vocab_size
        predicts = logits.argmax(dim=-1,keepdim=True) # B,1
        y = torch.cat((y,predicts),dim=-1)
    print(y.shape)
    for b in range(y.shape[0]):
        for i in y[b]:
            print(output_tokenizer.convert_id_to_token([int(i)]))

In [938]:
predict(transformer,'The rapid development of technology has brought about significant changes in our daily lives, making it more convenient for us to communicate and access information.')

torch.Size([1, 101])
['<bos>']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
['的']
